In [1]:
import pandas as pd 
import numpy as np

In [2]:
import os

In [3]:
import heapq

In [4]:
import json

# Creating  DataFrames based on The Next Month Prediction and The Precision

In [5]:
def clean_return(s):
    # Transform based on the following examples:
    #    stock1     Output
    # 1: "[-1.]" => "-1"
    # 2: "[1.]"  => "1"
    s.insert(1, "Next_Month", s.apply(lambda row : row["Stock"][row["Stock"].find("[") + 1:row["Stock"].find(".")], axis=1))
    # Change column type to int64 for column: 'clean_stock'
    s = s.astype({'Next_Month': 'int64'})
    s.drop(columns=['Stock'], inplace=True)
    return s

In [6]:
def preprossing_return(df: pd.DataFrame, name: str) -> pd.DataFrame:
    df.drop(columns=['Unnamed: 0'], inplace=True)
    df = df.T
    s = pd.DataFrame(df[4])
    s.rename(columns={4: f'Stock'}, inplace=True)
    s.reset_index(inplace=True)
    s = clean_return(s)
    s.rename(columns={'index' : 'Date'} , inplace=True)
    s = s.astype({'Date': 'datetime64[ns]'})
    s.rename(columns={'Next_Month' : name}, inplace=True)
    
    return s

In [7]:
def preprossing_precision(df: pd.DataFrame, name: str) -> pd.DataFrame:
    df.drop(columns=['Unnamed: 0'], inplace=True)
    df = df.T
    s = pd.DataFrame(df[1])
    s.rename(columns={1: f'{name}'}, inplace=True)
    s.reset_index(inplace=True)
    s.rename(columns={'index' : 'Date'} , inplace=True)
    s = s.astype({'Date': 'datetime64[ns]'})
    return s

# RF - RF

In [8]:
folder_path = './Final_result/RF-RF'     # Folder path

files = os.listdir(folder_path)

start_date = pd.to_datetime('2015-08').to_period('M')
end_date = pd.to_datetime('2023-07').to_period('M')

date_range1 = pd.period_range(start=start_date, end=end_date, freq='M')
date_range2 = pd.period_range(start=start_date, end=end_date, freq='M')

RfRf_next_month = pd.DataFrame(date_range1, columns=['Date'])
RfRf_precision =pd.DataFrame(date_range2, columns=['Date'])

RfRf_next_month = RfRf_next_month.astype({'Date': 'datetime64[ns]'})
RfRf_precision = RfRf_precision.astype({'Date': 'datetime64[ns]'})

for file in files :
    raw_data1 = pd.read_excel(f'{folder_path}/{file}')
    raw_data2 = pd.read_excel(f'{folder_path}/{file}')
    s = preprossing_return(raw_data1,file)
    RfRf_next_month = pd.merge(RfRf_next_month , s, how='outer', on='Date')


    d = preprossing_precision(raw_data2,file)
    RfRf_precision = pd.merge(RfRf_precision , d, how='outer', on='Date')


    RfRf_next_month = RfRf_next_month[:96]
    RfRf_precision = RfRf_precision[:96]

    RfRf_next_month.set_index('Date', inplace=True)
    RfRf_precision.set_index('Date', inplace=True)


stock_pool = {}
for inx in range(0, len(RfRf_next_month)):
    columns_with_values = []
    for name, item in RfRf_next_month.iloc[inx].items():
        if item > 0:
            columns_with_values.append((name, item))
    if columns_with_values:
        date_string = str(RfRf_next_month.index[inx].date())
        stock_pool[date_string] = columns_with_values


my_portfolio = {}
for idx in range(0, len(RfRf_precision)):
    date = RfRf_precision.index[idx].date()
    formatted_date = date.strftime('%Y-%m-%d')

    stock_ret = [x for x in stock_pool[str(date)]]
    stocks = [stock[0] for stock in stock_ret]

    columns_with_values = []
    for name, item in RfRf_precision.iloc[idx].items():
        if name in stocks:
            columns_with_values.append((name, item))
    sorted_list = sorted(columns_with_values, key=lambda x: x[1], reverse=True)
    if len(sorted_list) > 4:
        sorted_list = sorted_list[:4]
    else:
        sorted_list = sorted_list
    my_portfolio[formatted_date] = sorted_list


with open('./my_portfolio_RFRF.json', 'w') as f: # Save the portfolio
    json.dump(my_portfolio, f)






# RF- ANN

In [10]:
folder_path = './Final_result/RF-ANN'     # Folder path

files = os.listdir(folder_path)

start_date = pd.to_datetime('2015-08').to_period('M')
end_date = pd.to_datetime('2023-07').to_period('M')

date_range1 = pd.period_range(start=start_date, end=end_date, freq='M')
date_range2 = pd.period_range(start=start_date, end=end_date, freq='M')

RfRf_next_month = pd.DataFrame(date_range1, columns=['Date'])
RfRf_precision =pd.DataFrame(date_range2, columns=['Date'])

RfRf_next_month = RfRf_next_month.astype({'Date': 'datetime64[ns]'})
RfRf_precision = RfRf_precision.astype({'Date': 'datetime64[ns]'})

for file in files :
    raw_data1 = pd.read_excel(f'{folder_path}/{file}')
    raw_data2 = pd.read_excel(f'{folder_path}/{file}')
    s = preprossing_return(raw_data1,file)
    RfRf_next_month = pd.merge(RfRf_next_month , s, how='outer', on='Date')


    d = preprossing_precision(raw_data2,file)
    RfRf_precision = pd.merge(RfRf_precision , d, how='outer', on='Date')


    RfRf_next_month = RfRf_next_month[:96]
    RfRf_precision = RfRf_precision[:96]

    RfRf_next_month.set_index('Date', inplace=True)
    RfRf_precision.set_index('Date', inplace=True)


stock_pool = {}
for inx in range(0, len(RfRf_next_month)):
    columns_with_values = []
    for name, item in RfRf_next_month.iloc[inx].items():
        if item > 0:
            columns_with_values.append((name, item))
    if columns_with_values:
        date_string = str(RfRf_next_month.index[inx].date())
        stock_pool[date_string] = columns_with_values


my_portfolio = {}
for idx in range(0, len(RfRf_precision)):
    date = RfRf_precision.index[idx].date()
    formatted_date = date.strftime('%Y-%m-%d')

    stock_ret = [x for x in stock_pool[str(date)]]
    stocks = [stock[0] for stock in stock_ret]

    columns_with_values = []
    for name, item in RfRf_precision.iloc[idx].items():
        if name in stocks:
            columns_with_values.append((name, item))
    sorted_list = sorted(columns_with_values, key=lambda x: x[1], reverse=True)
    if len(sorted_list) > 4:
        sorted_list = sorted_list[:4]
    else:
        sorted_list = sorted_list
    my_portfolio[formatted_date] = sorted_list


with open('./my_portfolio_RFANN.json', 'w') as f: # Save the portfolio
    json.dump(my_portfolio, f)






In [11]:
my_portfolio

{'2015-08-01': [('RfAnn_فباهنر.xlsx', 0.6),
  ('RfAnn_شکبیر.xlsx', 0.5),
  ('RfAnn_شبهرن.xlsx', 0.4666666666666667),
  ('RfAnn_خساپا.xlsx', 0.45)],
 '2015-09-01': [('RfAnn_فولاژ.xlsx', 0.6),
  ('RfAnn_خپارس.xlsx', 0.4),
  ('RfAnn_فولاد.xlsx', 0.4),
  ('RfAnn_شرانول.xlsx', 0.3666666666666666)],
 '2015-10-01': [('RfAnn_شبهرن.xlsx', 1),
  ('RfAnn_فباهنر.xlsx', 0.8),
  ('RfAnn_شخارک.xlsx', 0.7),
  ('RfAnn_خساپا.xlsx', 0.6666666666666666)],
 '2015-11-01': [('RfAnn_خساپا.xlsx', 0.6666666666666666),
  ('RfAnn_شفن.xlsx', 0.6666666666666666),
  ('RfAnn_شاوان.xlsx', 0.5833333333333334),
  ('RfAnn_کرماشا.xlsx', 0.5833333333333334)],
 '2015-12-01': [('RfAnn_فباهنر.xlsx', 0.8333333333333334),
  ('RfAnn_شاراک.xlsx', 0.7222222222222222),
  ('RfAnn_ذوب.xlsx', 0.6666666666666666),
  ('RfAnn_شپنا.xlsx', 0.6666666666666666)],
 '2016-01-01': [('RfAnn_خزامیاد.xlsx', 0.875),
  ('RfAnn_زاگرس.xlsx', 0.8333333333333334),
  ('RfAnn_فایرا.xlsx', 0.8333333333333334),
  ('RfAnn_شخارک.xlsx', 0.7222222222222222)],
 

# RF - SVM

In [12]:
folder_path = './Final_result/RF-SVM'     # Folder path

files = os.listdir(folder_path)

start_date = pd.to_datetime('2015-08').to_period('M')
end_date = pd.to_datetime('2023-07').to_period('M')

date_range1 = pd.period_range(start=start_date, end=end_date, freq='M')
date_range2 = pd.period_range(start=start_date, end=end_date, freq='M')

RfRf_next_month = pd.DataFrame(date_range1, columns=['Date'])
RfRf_precision =pd.DataFrame(date_range2, columns=['Date'])

RfRf_next_month = RfRf_next_month.astype({'Date': 'datetime64[ns]'})
RfRf_precision = RfRf_precision.astype({'Date': 'datetime64[ns]'})

for file in files :
    raw_data1 = pd.read_excel(f'{folder_path}/{file}')
    raw_data2 = pd.read_excel(f'{folder_path}/{file}')
    s = preprossing_return(raw_data1,file)
    RfRf_next_month = pd.merge(RfRf_next_month , s, how='outer', on='Date')


    d = preprossing_precision(raw_data2,file)
    RfRf_precision = pd.merge(RfRf_precision , d, how='outer', on='Date')


    RfRf_next_month = RfRf_next_month[:96]
    RfRf_precision = RfRf_precision[:96]

    RfRf_next_month.set_index('Date', inplace=True)
    RfRf_precision.set_index('Date', inplace=True)


stock_pool = {}
for inx in range(0, len(RfRf_next_month)):
    columns_with_values = []
    for name, item in RfRf_next_month.iloc[inx].items():
        if item > 0:
            columns_with_values.append((name, item))
    if columns_with_values:
        date_string = str(RfRf_next_month.index[inx].date())
        stock_pool[date_string] = columns_with_values


my_portfolio = {}
for idx in range(0, len(RfRf_precision)):
    date = RfRf_precision.index[idx].date()
    formatted_date = date.strftime('%Y-%m-%d')

    stock_ret = [x for x in stock_pool[str(date)]]
    stocks = [stock[0] for stock in stock_ret]

    columns_with_values = []
    for name, item in RfRf_precision.iloc[idx].items():
        if name in stocks:
            columns_with_values.append((name, item))
    sorted_list = sorted(columns_with_values, key=lambda x: x[1], reverse=True)
    if len(sorted_list) > 4:
        sorted_list = sorted_list[:4]
    else:
        sorted_list = sorted_list
    my_portfolio[formatted_date] = sorted_list


with open('./my_portfolio_RFSVM.json', 'w') as f: # Save the portfolio
    json.dump(my_portfolio, f)






In [13]:
my_portfolio

{'2015-08-01': [('RfSvm_شپدیس.xlsx', 0.8),
  ('RfSvm_فاسمین.xlsx', 0.5333333333333333),
  ('RfSvm_شنفت.xlsx', 0.4),
  ('RfSvm_شکبیر.xlsx', 0.16)],
 '2015-09-01': [('RfSvm_شپدیس.xlsx', 0.4666666666666667),
  ('RfSvm_مارون.xlsx', 0.1),
  ('RfSvm_جم.xlsx', 0.06666666666666667),
  ('RfSvm_خراسان.xlsx', 0)],
 '2015-10-01': [('RfSvm_شبریز.xlsx', 0.6),
  ('RfSvm_مارون.xlsx', 0.3666666666666666),
  ('RfSvm_جم.xlsx', 0.04),
  ('RfSvm_شنفت.xlsx', 0)],
 '2015-11-01': [('RfSvm_شپنا.xlsx', 0.6333333333333333),
  ('RfSvm_شراز.xlsx', 0.5),
  ('RfSvm_جم.xlsx', 0.1666666666666667)],
 '2015-12-01': [('RfSvm_خودرو.xlsx', 0.6666666666666666),
  ('RfSvm_مارون.xlsx', 0.5),
  ('RfSvm_فایرا.xlsx', 0.4166666666666667),
  ('RfSvm_جم.xlsx', 0.08333333333333333)],
 '2016-01-01': [('RfSvm_شپنا.xlsx', 0.4722222222222222),
  ('RfSvm_شپدیس.xlsx', 0.2222222222222222),
  ('RfSvm_شکبیر.xlsx', 0.1944444444444444),
  ('RfSvm_خراسان.xlsx', 0)],
 '2016-02-01': [('RfSvm_شکبیر.xlsx', 0.2222222222222222),
  ('RfSvm_شپنا.xlsx',

# RF - XGB

In [14]:
folder_path = './Final_result/RF-XGB'     # Folder path

files = os.listdir(folder_path)

start_date = pd.to_datetime('2015-08').to_period('M')
end_date = pd.to_datetime('2023-07').to_period('M')

date_range1 = pd.period_range(start=start_date, end=end_date, freq='M')
date_range2 = pd.period_range(start=start_date, end=end_date, freq='M')

RfRf_next_month = pd.DataFrame(date_range1, columns=['Date'])
RfRf_precision =pd.DataFrame(date_range2, columns=['Date'])

RfRf_next_month = RfRf_next_month.astype({'Date': 'datetime64[ns]'})
RfRf_precision = RfRf_precision.astype({'Date': 'datetime64[ns]'})

for file in files :
    raw_data1 = pd.read_excel(f'{folder_path}/{file}')
    raw_data2 = pd.read_excel(f'{folder_path}/{file}')
    s = preprossing_return(raw_data1,file)
    RfRf_next_month = pd.merge(RfRf_next_month , s, how='outer', on='Date')


    d = preprossing_precision(raw_data2,file)
    RfRf_precision = pd.merge(RfRf_precision , d, how='outer', on='Date')


    RfRf_next_month = RfRf_next_month[:96]
    RfRf_precision = RfRf_precision[:96]

    RfRf_next_month.set_index('Date', inplace=True)
    RfRf_precision.set_index('Date', inplace=True)


stock_pool = {}
for inx in range(0, len(RfRf_next_month)):
    columns_with_values = []
    for name, item in RfRf_next_month.iloc[inx].items():
        if item > 0:
            columns_with_values.append((name, item))
    if columns_with_values:
        date_string = str(RfRf_next_month.index[inx].date())
        stock_pool[date_string] = columns_with_values


my_portfolio = {}
for idx in range(0, len(RfRf_precision)):
    date = RfRf_precision.index[idx].date()
    formatted_date = date.strftime('%Y-%m-%d')

    stock_ret = [x for x in stock_pool[str(date)]]
    stocks = [stock[0] for stock in stock_ret]

    columns_with_values = []
    for name, item in RfRf_precision.iloc[idx].items():
        if name in stocks:
            columns_with_values.append((name, item))
    sorted_list = sorted(columns_with_values, key=lambda x: x[1], reverse=True)
    if len(sorted_list) > 4:
        sorted_list = sorted_list[:4]
    else:
        sorted_list = sorted_list
    my_portfolio[formatted_date] = sorted_list


with open('./my_portfolio_RFXGB.json', 'w') as f: # Save the portfolio
    json.dump(my_portfolio, f)






In [15]:
my_portfolio

{'2015-08-01': [('RfXgb_شراز.xlsx', 1),
  ('RfXgb_شبهرن.xlsx', 1),
  ('RfXgb_شبریز.xlsx', 1),
  ('RfXgb_شفن.xlsx', 1)],
 '2015-09-01': [('RfXgb_شراز.xlsx', 1),
  ('RfXgb_شبهرن.xlsx', 1),
  ('RfXgb_شپدیس.xlsx', 0.8666666666666666),
  ('RfXgb_خساپا.xlsx', 0.6)],
 '2015-10-01': [('RfXgb_شیراز.xlsx', 0.8666666666666666),
  ('RfXgb_شپدیس.xlsx', 0.8666666666666666),
  ('RfXgb_خراسان.xlsx', 0.8666666666666666),
  ('RfXgb_شبهرن.xlsx', 0.85)],
 '2015-11-01': [('RfXgb_خودرو.xlsx', 0.8888888888888888),
  ('RfXgb_شپدیس.xlsx', 0.8333333333333334),
  ('RfXgb_ذوب.xlsx', 0.7999999999999999),
  ('RfXgb_شفن.xlsx', 0.6666666666666666)],
 '2015-12-01': [('RfXgb_فولاژ.xlsx', 1),
  ('RfXgb_کگل.xlsx', 1),
  ('RfXgb_شیراز.xlsx', 0.875),
  ('RfXgb_شفن.xlsx', 0.8666666666666667)],
 '2016-01-01': [('RfXgb_کنور.xlsx', 0.9166666666666666),
  ('RfXgb_شکبیر.xlsx', 0.8333333333333334),
  ('RfXgb_جم.xlsx', 0.8333333333333334),
  ('RfXgb_فباهنر.xlsx', 0.7999999999999999)],
 '2016-02-01': [('RfXgb_کنور.xlsx', 1),
  ('Rf

# Lasso Logestic Regression - ANN

In [16]:
folder_path = './Final_result/LassoLogReg_ANN'     # Folder path

files = os.listdir(folder_path)

start_date = pd.to_datetime('2015-08').to_period('M')
end_date = pd.to_datetime('2023-07').to_period('M')

date_range1 = pd.period_range(start=start_date, end=end_date, freq='M')
date_range2 = pd.period_range(start=start_date, end=end_date, freq='M')

RfRf_next_month = pd.DataFrame(date_range1, columns=['Date'])
RfRf_precision =pd.DataFrame(date_range2, columns=['Date'])

RfRf_next_month = RfRf_next_month.astype({'Date': 'datetime64[ns]'})
RfRf_precision = RfRf_precision.astype({'Date': 'datetime64[ns]'})

for file in files :
    raw_data1 = pd.read_excel(f'{folder_path}/{file}')
    raw_data2 = pd.read_excel(f'{folder_path}/{file}')
    s = preprossing_return(raw_data1,file)
    RfRf_next_month = pd.merge(RfRf_next_month , s, how='outer', on='Date')


    d = preprossing_precision(raw_data2,file)
    RfRf_precision = pd.merge(RfRf_precision , d, how='outer', on='Date')


    RfRf_next_month = RfRf_next_month[:96]
    RfRf_precision = RfRf_precision[:96]

    RfRf_next_month.set_index('Date', inplace=True)
    RfRf_precision.set_index('Date', inplace=True)


stock_pool = {}
for inx in range(0, len(RfRf_next_month)):
    columns_with_values = []
    for name, item in RfRf_next_month.iloc[inx].items():
        if item > 0:
            columns_with_values.append((name, item))
    if columns_with_values:
        date_string = str(RfRf_next_month.index[inx].date())
        stock_pool[date_string] = columns_with_values


my_portfolio = {}
for idx in range(0, len(RfRf_precision)):
    date = RfRf_precision.index[idx].date()
    formatted_date = date.strftime('%Y-%m-%d')

    stock_ret = [x for x in stock_pool[str(date)]]
    stocks = [stock[0] for stock in stock_ret]

    columns_with_values = []
    for name, item in RfRf_precision.iloc[idx].items():
        if name in stocks:
            columns_with_values.append((name, item))
    sorted_list = sorted(columns_with_values, key=lambda x: x[1], reverse=True)
    if len(sorted_list) > 4:
        sorted_list = sorted_list[:4]
    else:
        sorted_list = sorted_list
    my_portfolio[formatted_date] = sorted_list


with open('./my_portfolio_LassoLogRegANN.json', 'w') as f: # Save the portfolio
    json.dump(my_portfolio, f)






In [17]:
my_portfolio

{'2015-08-01': [('LassoLogRegAnn_فباهنر.xlsx', 0.4),
  ('LassoLogRegAnn_خپارس.xlsx', 0.4),
  ('LassoLogRegAnn_ذوب.xlsx', 0.3),
  ('LassoLogRegAnn_خزامیاد.xlsx', 0.3)],
 '2015-09-01': [('LassoLogRegAnn_فولاژ.xlsx', 1),
  ('LassoLogRegAnn_شنفت.xlsx', 1),
  ('LassoLogRegAnn_شخارک.xlsx', 0.8),
  ('LassoLogRegAnn_شبندر.xlsx', 0.7)],
 '2015-10-01': [('LassoLogRegAnn_شراز.xlsx', 0.6666666666666667),
  ('LassoLogRegAnn_فاسمین.xlsx', 0.3666666666666666),
  ('LassoLogRegAnn_شیراز.xlsx', 0.3),
  ('LassoLogRegAnn_شاوان.xlsx', 0.2666666666666667)],
 '2015-11-01': [('LassoLogRegAnn_فاسمین.xlsx', 0.8333333333333334),
  ('LassoLogRegAnn_شاراک.xlsx', 0.8055555555555555),
  ('LassoLogRegAnn_شبهرن.xlsx', 0.6666666666666666),
  ('LassoLogRegAnn_شپنا.xlsx', 0.6333333333333333)],
 '2015-12-01': [('LassoLogRegAnn_زاگرس.xlsx', 1),
  ('LassoLogRegAnn_شتران.xlsx', 0.6333333333333333),
  ('LassoLogRegAnn_مارون.xlsx', 0.5555555555555556),
  ('LassoLogRegAnn_فولاد.xlsx', 0.5333333333333333)],
 '2016-01-01': [('Las

# Lasso Logestic Regression - RF

In [18]:
folder_path = './Final_result/LassoLogReg_RF'     # Folder path

files = os.listdir(folder_path)

start_date = pd.to_datetime('2015-08').to_period('M')
end_date = pd.to_datetime('2023-07').to_period('M')

date_range1 = pd.period_range(start=start_date, end=end_date, freq='M')
date_range2 = pd.period_range(start=start_date, end=end_date, freq='M')

RfRf_next_month = pd.DataFrame(date_range1, columns=['Date'])
RfRf_precision =pd.DataFrame(date_range2, columns=['Date'])

RfRf_next_month = RfRf_next_month.astype({'Date': 'datetime64[ns]'})
RfRf_precision = RfRf_precision.astype({'Date': 'datetime64[ns]'})

for file in files :
    raw_data1 = pd.read_excel(f'{folder_path}/{file}')
    raw_data2 = pd.read_excel(f'{folder_path}/{file}')
    s = preprossing_return(raw_data1,file)
    RfRf_next_month = pd.merge(RfRf_next_month , s, how='outer', on='Date')


    d = preprossing_precision(raw_data2,file)
    RfRf_precision = pd.merge(RfRf_precision , d, how='outer', on='Date')


    RfRf_next_month = RfRf_next_month[:96]
    RfRf_precision = RfRf_precision[:96]

    RfRf_next_month.set_index('Date', inplace=True)
    RfRf_precision.set_index('Date', inplace=True)


stock_pool = {}
for inx in range(0, len(RfRf_next_month)):
    columns_with_values = []
    for name, item in RfRf_next_month.iloc[inx].items():
        if item > 0:
            columns_with_values.append((name, item))
    if columns_with_values:
        date_string = str(RfRf_next_month.index[inx].date())
        stock_pool[date_string] = columns_with_values


my_portfolio = {}
for idx in range(0, len(RfRf_precision)):
    date = RfRf_precision.index[idx].date()
    formatted_date = date.strftime('%Y-%m-%d')

    stock_ret = [x for x in stock_pool[str(date)]]
    stocks = [stock[0] for stock in stock_ret]

    columns_with_values = []
    for name, item in RfRf_precision.iloc[idx].items():
        if name in stocks:
            columns_with_values.append((name, item))
    sorted_list = sorted(columns_with_values, key=lambda x: x[1], reverse=True)
    if len(sorted_list) > 4:
        sorted_list = sorted_list[:4]
    else:
        sorted_list = sorted_list
    my_portfolio[formatted_date] = sorted_list


with open('./my_portfolio_LassoLogRegRF.json', 'w') as f: # Save the portfolio
    json.dump(my_portfolio, f)






In [19]:
my_portfolio

{'2015-08-01': [('LassoLogRegRF_شبریز.xlsx', 0.45),
  ('LassoLogRegRF_خپارس.xlsx', 0.2666666666666667),
  ('LassoLogRegRF_شبندر.xlsx', 0)],
 '2015-09-01': [('LassoLogRegRF_فاسمین.xlsx', 0.8666666666666666),
  ('LassoLogRegRF_شرانول.xlsx', 0.6),
  ('LassoLogRegRF_ذوب.xlsx', 0.6),
  ('LassoLogRegRF_شپدیس.xlsx', 0.4666666666666667)],
 '2015-10-01': [('LassoLogRegRF_خپارس.xlsx', 0.7),
  ('LassoLogRegRF_فخوز.xlsx', 0.6),
  ('LassoLogRegRF_شپنا.xlsx', 0.5333333333333333),
  ('LassoLogRegRF_شیراز.xlsx', 0.36)],
 '2015-11-01': [('LassoLogRegRF_خراسان.xlsx', 0.4166666666666667),
  ('LassoLogRegRF_مارون.xlsx', 0),
  ('LassoLogRegRF_شپدیس.xlsx', 0)],
 '2015-12-01': [('LassoLogRegRF_شیران.xlsx', 0.6666666666666666),
  ('LassoLogRegRF_شپدیس.xlsx', 0.5),
  ('LassoLogRegRF_شاراک.xlsx', 0.4166666666666667),
  ('LassoLogRegRF_ذوب.xlsx', 0.4166666666666667)],
 '2016-01-01': [('LassoLogRegRF_شبریز.xlsx', 0.6666666666666666),
  ('LassoLogRegRF_خزامیاد.xlsx', 0.6666666666666666),
  ('LassoLogRegRF_شپدیس.xl

# Lasso Logestic Regression - XGB

In [20]:
folder_path = './Final_result/LassoLogReg_XGB'     # Folder path

files = os.listdir(folder_path)

start_date = pd.to_datetime('2015-08').to_period('M')
end_date = pd.to_datetime('2023-07').to_period('M')

date_range1 = pd.period_range(start=start_date, end=end_date, freq='M')
date_range2 = pd.period_range(start=start_date, end=end_date, freq='M')

RfRf_next_month = pd.DataFrame(date_range1, columns=['Date'])
RfRf_precision =pd.DataFrame(date_range2, columns=['Date'])

RfRf_next_month = RfRf_next_month.astype({'Date': 'datetime64[ns]'})
RfRf_precision = RfRf_precision.astype({'Date': 'datetime64[ns]'})

for file in files :
    raw_data1 = pd.read_excel(f'{folder_path}/{file}')
    raw_data2 = pd.read_excel(f'{folder_path}/{file}')
    s = preprossing_return(raw_data1,file)
    RfRf_next_month = pd.merge(RfRf_next_month , s, how='outer', on='Date')


    d = preprossing_precision(raw_data2,file)
    RfRf_precision = pd.merge(RfRf_precision , d, how='outer', on='Date')


    RfRf_next_month = RfRf_next_month[:96]
    RfRf_precision = RfRf_precision[:96]

    RfRf_next_month.set_index('Date', inplace=True)
    RfRf_precision.set_index('Date', inplace=True)


stock_pool = {}
for inx in range(0, len(RfRf_next_month)):
    columns_with_values = []
    for name, item in RfRf_next_month.iloc[inx].items():
        if item > 0:
            columns_with_values.append((name, item))
    if columns_with_values:
        date_string = str(RfRf_next_month.index[inx].date())
        stock_pool[date_string] = columns_with_values


my_portfolio = {}
for idx in range(0, len(RfRf_precision)):
    date = RfRf_precision.index[idx].date()
    formatted_date = date.strftime('%Y-%m-%d')

    stock_ret = [x for x in stock_pool[str(date)]]
    stocks = [stock[0] for stock in stock_ret]

    columns_with_values = []
    for name, item in RfRf_precision.iloc[idx].items():
        if name in stocks:
            columns_with_values.append((name, item))
    sorted_list = sorted(columns_with_values, key=lambda x: x[1], reverse=True)
    if len(sorted_list) > 4:
        sorted_list = sorted_list[:4]
    else:
        sorted_list = sorted_list
    my_portfolio[formatted_date] = sorted_list


with open('./my_portfolio_LassoLogRegXGB.json', 'w') as f: # Save the portfolio
    json.dump(my_portfolio, f)






In [21]:
my_portfolio

{'2015-08-01': [('LassoLogRegXGB_شبریز.xlsx', 1),
  ('LassoLogRegXGB_کچاد.xlsx', 1),
  ('LassoLogRegXGB_شاوان.xlsx', 0.8666666666666666),
  ('LassoLogRegXGB_خراسان.xlsx', 0.6)],
 '2015-09-01': [('LassoLogRegXGB_فباهنر.xlsx', 1),
  ('LassoLogRegXGB_زاگرس.xlsx', 1),
  ('LassoLogRegXGB_شیران.xlsx', 1),
  ('LassoLogRegXGB_شبهرن.xlsx', 0.8666666666666666)],
 '2015-10-01': [('LassoLogRegXGB_کگل.xlsx', 1),
  ('LassoLogRegXGB_شیراز.xlsx', 1),
  ('LassoLogRegXGB_خراسان.xlsx', 0.8666666666666666),
  ('LassoLogRegXGB_کچاد.xlsx', 0.85)],
 '2015-11-01': [('LassoLogRegXGB_شفن.xlsx', 1),
  ('LassoLogRegXGB_مارون.xlsx', 0.8888888888888888),
  ('LassoLogRegXGB_شپدیس.xlsx', 0.875),
  ('LassoLogRegXGB_شیراز.xlsx', 0.8666666666666667)],
 '2015-12-01': [('LassoLogRegXGB_شیران.xlsx', 0.8888888888888888),
  ('LassoLogRegXGB_شپدیس.xlsx', 0.8888888888888888),
  ('LassoLogRegXGB_شبهرن.xlsx', 0.8888888888888888),
  ('LassoLogRegXGB_مارون.xlsx', 0.8888888888888888)],
 '2016-01-01': [('LassoLogRegXGB_شکبیر.xlsx', 

# Lasso Logestic Regression - SVM

In [22]:
folder_path = './Final_result/LassoLogReg_SVM'     # Folder path

files = os.listdir(folder_path)

start_date = pd.to_datetime('2015-08').to_period('M')
end_date = pd.to_datetime('2023-07').to_period('M')

date_range1 = pd.period_range(start=start_date, end=end_date, freq='M')
date_range2 = pd.period_range(start=start_date, end=end_date, freq='M')

RfRf_next_month = pd.DataFrame(date_range1, columns=['Date'])
RfRf_precision =pd.DataFrame(date_range2, columns=['Date'])

RfRf_next_month = RfRf_next_month.astype({'Date': 'datetime64[ns]'})
RfRf_precision = RfRf_precision.astype({'Date': 'datetime64[ns]'})

for file in files :
    raw_data1 = pd.read_excel(f'{folder_path}/{file}')
    raw_data2 = pd.read_excel(f'{folder_path}/{file}')
    s = preprossing_return(raw_data1,file)
    RfRf_next_month = pd.merge(RfRf_next_month , s, how='outer', on='Date')


    d = preprossing_precision(raw_data2,file)
    RfRf_precision = pd.merge(RfRf_precision , d, how='outer', on='Date')


    RfRf_next_month = RfRf_next_month[:96]
    RfRf_precision = RfRf_precision[:96]

    RfRf_next_month.set_index('Date', inplace=True)
    RfRf_precision.set_index('Date', inplace=True)


stock_pool = {}
for inx in range(0, len(RfRf_next_month)):
    columns_with_values = []
    for name, item in RfRf_next_month.iloc[inx].items():
        if item > 0:
            columns_with_values.append((name, item))
    if columns_with_values:
        date_string = str(RfRf_next_month.index[inx].date())
        stock_pool[date_string] = columns_with_values


my_portfolio = {}
for idx in range(0, len(RfRf_precision)):
    date = RfRf_precision.index[idx].date()
    formatted_date = date.strftime('%Y-%m-%d')

    stock_ret = [x for x in stock_pool[str(date)]]
    stocks = [stock[0] for stock in stock_ret]

    columns_with_values = []
    for name, item in RfRf_precision.iloc[idx].items():
        if name in stocks:
            columns_with_values.append((name, item))
    sorted_list = sorted(columns_with_values, key=lambda x: x[1], reverse=True)
    if len(sorted_list) > 4:
        sorted_list = sorted_list[:4]
    else:
        sorted_list = sorted_list
    my_portfolio[formatted_date] = sorted_list


with open('./my_portfolio_LassoLogRegSVM.json', 'w') as f: # Save the portfolio
    json.dump(my_portfolio, f)






In [23]:
my_portfolio

{'2015-08-01': [('LassoLogRegSvm_شکبیر.xlsx', 0.6666666666666667),
  ('LassoLogRegSvm_شبریز.xlsx', 0.5333333333333333),
  ('LassoLogRegSvm_خودرو.xlsx', 0.2),
  ('LassoLogRegSvm_فایرا.xlsx', 0.1333333333333333)],
 '2015-09-01': [('LassoLogRegSvm_شپنا.xlsx', 0.65),
  ('LassoLogRegSvm_شکبیر.xlsx', 0.4),
  ('LassoLogRegSvm_خودرو.xlsx', 0.36),
  ('LassoLogRegSvm_شخارک.xlsx', 0.2)],
 '2015-10-01': [('LassoLogRegSvm_ارفع.xlsx', 1),
  ('LassoLogRegSvm_شکبیر.xlsx', 0.6),
  ('LassoLogRegSvm_خساپا.xlsx', 0.4),
  ('LassoLogRegSvm_شپنا.xlsx', 0.3)],
 '2015-11-01': [('LassoLogRegSvm_شخارک.xlsx', 0.6666666666666666),
  ('LassoLogRegSvm_خودرو.xlsx', 0.5555555555555556),
  ('LassoLogRegSvm_شکبیر.xlsx', 0.4666666666666666),
  ('LassoLogRegSvm_ارفع.xlsx', 0.3333333333333333)],
 '2015-12-01': [('LassoLogRegSvm_جم.xlsx', 1),
  ('LassoLogRegSvm_شفن.xlsx', 0.7222222222222222),
  ('LassoLogRegSvm_شپنا.xlsx', 0.7222222222222222),
  ('LassoLogRegSvm_شبندر.xlsx', 0.3333333333333333)],
 '2016-01-01': [('LassoLogR